In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import maxabs_scale, MaxAbsScaler
import anndata
import os
import sys
# from model.stDiff_train import normal_train_stDiff
sys.path.append('../..')
from model.stDiff_model import DiT_stDiff
# from process.data import * 


ModuleNotFoundError: No module named 'ray'

In [7]:
spatial_data_path = '/Users/derekebowman/Coding Projects/stDiff/stDiff/DEREK/LiverFibrosisData/adata_healthy_diseased_merfish.h5ad'

sequencing_data_path ='/Users/derekebowman/Coding Projects/stDiff/stDiff/DEREK/LiverFibrosisData/adata_healthy_diseased_nucseq.h5ad'

In [8]:
adata_spatial = sc.read_h5ad(spatial_data_path)
adata_seq = sc.read_h5ad(sequencing_data_path)


In [9]:
adata_spatial.var_names

Index(['A2M', 'ABCA1', 'ABCA13', 'ABCB1', 'ABCB11', 'ABCC2', 'ABLIM1',
       'AC007262.2', 'AC021242.2', 'ACMSD',
       ...
       'TTC7B', 'TXNRD1', 'UGT2B10', 'UGT2B4', 'VMP1', 'VSIG4', 'XDH',
       'ZC3H13', 'ZNF483', 'ZSWIM6'],
      dtype='object', length=317)

In [10]:
adata_seq.var_names

Index(['ENSG00000238009', 'ENSG00000268903', 'ENSG00000241860',
       'ENSG00000228463', 'ENSG00000290385', 'ENSG00000230021',
       'ENSG00000291215', 'LINC01409', 'LINC00115', 'LINC01128',
       ...
       'ENSG00000230392', 'ENSG00000226661', 'STAG2-AS1', 'SH2D1A',
       'LINC00629', 'PRRG3', 'LAGE3', 'ENSG00000224216', 'ENSG00000273906',
       'MED14P1'],
      dtype='object', length=24619)

In [11]:
shared_genes = adata_seq.var_names.intersection(adata_spatial.var_names)
adata_sc = adata_seq[:, shared_genes]
adata_sp = adata_spatial[:, shared_genes]

# adata_sc.write("my_sequencing_data_shared_genes.h5ad")
#adata_sp.write("my_spatial_data_shared_genes.h5ad")

In [12]:
def data_augment(adata,  fixed: bool, noise_std):
   
    noise_stddev = noise_std
    augmented_adata = adata.copy()
    gene_expression = adata.X
    
    if fixed: 
        augmented_adata.X = augmented_adata.X + np.full(gene_expression.shape, noise_stddev)
    else:
        augmented_adata.X = augmented_adata.X + np.abs(np.random.normal(0, noise_stddev, gene_expression.shape))   
    
    merge_adata = adata.concatenate(augmented_adata, join='outer')
    
    
    return merge_adata
    

In [13]:
def scale(adata):
    scaler = MaxAbsScaler()
    normalized_data = scaler.fit_transform(adata.X.T).T
    adata.X = normalized_data
    return adata

In [14]:
shared_genes_spatial_path = '/Users/derekebowman/Coding Projects/stDiff/stDiff/DEREK/LiverFibrosisData/my_spatial_data_shared_genes.h5ad'
shared_genes_sequencing_path = '/Users/derekebowman/Coding Projects/stDiff/stDiff/DEREK/LiverFibrosisData/my_sequencing_data_shared_genes.h5ad'

In [15]:
adata_sequencing_shared= sc.read_h5ad(shared_genes_sequencing_path)
adata_spatial_shared = sc.read_h5ad(shared_genes_spatial_path)

In [16]:
adata_seq2 = data_augment(adata_sequencing_shared.copy(), True, noise_std=10)
adata_spatial2 = adata_spatial_shared.copy()

sc.pp.normalize_total(adata_seq2, target_sum=1e4)
sc.pp.log1p(adata_seq2)
adata_seq2 = scale(adata_seq2) # stDiff need
data_seq_array = adata_seq2.X

sc.pp.normalize_total(adata_spatial2, target_sum=1e4)
sc.pp.log1p(adata_spatial2)
adata_spatial2 = scale(adata_spatial2)
data_spatial_array = adata_spatial2.X

sp_genes = np.array(adata_spatial2.var_names)
sp_data = pd.DataFrame(data=data_spatial_array, columns=sp_genes)
sc_data = pd.DataFrame(data=data_seq_array, columns=sp_genes)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/merge.py:847: FutureWarning: The default value of 'ignore' for the `na_action` parameter in pandas.Categorical.map is deprecated and will be changed to 'None' in a future version. Please set na_action to the desired value to avoid seeing this warning
  concat_indices = concat_indices.str.cat(label_col.map(str), sep=index_unique)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/merge.py:847: FutureWarning: The default value of 'ignore' for the `na_action` parameter in pandas.Categorical.map is deprecated and will be changed to 'None' in a future version. Please set na_action to the desired value to avoid seeing this warning
  concat_indices = concat_indices.str.cat(label_col.map(str), sep=index_unique)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted 

In [17]:
sp_data

,MIR34AHG,SLC25A33,STMN1,MFSD2A,CTPS1,SLFNL1,ECHDC2,SCP2,PLPP3,LEPR,...,VSIG4,TENM1,CD5L,AKR1B10,SCD,GATA1,ALAS2,CD8A,CD68,TBX21
0,0.513306,0.000000,0.513306,0.513306,0.000000,0.513306,0.857922,0.0,0.513306,0.0,...,0.000000,0.000000,0.000000,0.513306,0.747640,0.000000,0.0,0.716956,0.000000,0.676218
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.875185,0.0,0.875185,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.639469,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.639469,0.000000,0.736251,0.000000,0.870800,0.000000,0.0,0.000000,0.000000,0.639469
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.944683,0.000000,0.000000,0.000000,0.0,0.724891,0.849329,0.000000
4,0.000000,0.620786,0.000000,0.000000,0.716296,0.620786,0.716296,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.620786,0.829525,0.716296,0.0,0.620786,0.620786,0.620786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575706,0.000000,0.000000,0.000000,0.000000,0.933522,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.933522,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
575707,0.000000,0.954444,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
575708,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
575709,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


# Method


In [18]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import numpy as np
import math

In [19]:
def get_data_loader(data_ary:np.ndarray, 
                    cell_type:np.ndarray, 
                    batch_size:int=512,
                    is_shuffle:bool=True,
                    ):
       

        data_tensor = torch.from_numpy(data_ary.astype(np.float32))
        cell_type_tensor = torch.from_numpy(cell_type.astype(np.float32))
        dataset = TensorDataset(data_tensor,cell_type_tensor)
        generator = torch.Generator(device='cpu')
        return DataLoader(
                dataset, batch_size=batch_size, shuffle=is_shuffle, drop_last=False , generator=generator) #, generator=torch.Generator(device = 'cuda') 

In [20]:
def modulate(x, shift, scale):

    res = x * (1 + scale) + shift
    return res

In [23]:
def normal_train_stDiff(model,
                 dataloader,
                 lr: float = 1e-4,
                 num_epoch: int = 1400,
                 pred_type: str = 'noise',
                 diffusion_step: int = 1000,
                 device=torch.device('cuda:1'),
                 is_tqdm: bool = True,
                 is_tune: bool = False,
                 mask = None):
    """

    Args:
        lr (float): learning rate 
        pred_type (str, optional): noise or x_0. Defaults to 'noise'.
        diffusion_step (int, optional): timestep. Defaults to 1000.
        device (_type_, optional): Defaults to torch.device('cuda:1').
        is_tqdm (bool, optional): tqdm. Defaults to True.
        is_tune (bool, optional):  ray tune. Defaults to False.

    Raises:
        NotImplementedError: _description_
    """

    noise_scheduler = NoiseScheduler(
        num_timesteps=diffusion_step,
        beta_schedule='cosine'
    )

    criterion = nn.MSELoss()
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0)

    if is_tqdm:
        t_epoch = tqdm(range(num_epoch), ncols=100)
    else:
        t_epoch = range(num_epoch)

    model.train()

    for epoch in t_epoch:
        epoch_loss = 0.
        for i, (x, x_cond) in enumerate(dataloader): 
            x, x_cond = x.float().to(device), x_cond.float().to(device)
            # celltype = celltype.to(device)

            noise = torch.randn(x.shape).to(device)
            timesteps = torch.randint(1, diffusion_step, (x.shape[0],)).long()

            x_t = noise_scheduler.add_noise(x,
                                            noise,
                                            timesteps=timesteps)

            mask = torch.tensor(mask).to(device)
            x_noisy = x_t * (1 - mask) + x * mask

            noise_pred = model(x_noisy, t=timesteps.to(device), y=x_cond) 
            # loss = criterion(noise_pred, noise)

            loss = criterion(noise*(1-mask), noise_pred*(1-mask))

            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # type: ignore
            optimizer.step()
            optimizer.zero_grad()
            epoch_loss += loss.item()

        epoch_loss = epoch_loss / (i + 1)  # type: ignore
        if is_tqdm:
            t_epoch.set_postfix_str(f'{pred_type} loss:{epoch_loss:.5f}')  # type: ignore
        if is_tune:
            session.report({'loss': epoch_loss})


In [24]:

lr = 0.00016046744893538737 
depth = 6 
num_epoch = 900 
diffusion_step = 1500 
batch_size = 2048 
hidden_size = 512 
head = 16

# mask
cell_num = data_spatial_array.shape[0]
gene_num = data_spatial_array.shape[1]
mask = np.ones((gene_num,), dtype='float32')

# gene_id_test
train_size = 0.8
gene_names_rnaseq = sp_genes 
np.random.seed(0)
n_genes = len(gene_names_rnaseq)
gene_ids_train = sorted(
    np.random.choice(range(n_genes), int(n_genes * train_size), False)
)
gene_ids_test = sorted(set(range(n_genes)) - set(gene_ids_train)) # test

mask[gene_ids_test] = 0

seq = data_seq_array
st = data_spatial_array
data_seq_masked = seq * mask
data_spatial_masked = st * mask

seq = seq * 2 - 1
data_seq_masked = data_seq_masked * 2 - 1

st = st * 2 - 1
data_spatial_masked = data_spatial_masked * 2 - 1

dataloader = get_data_loader(
    seq, # all gene
    data_seq_masked, # test gene = 0
    batch_size=batch_size, 
    is_shuffle=True)

seed = 1202
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

model = DiT_stDiff(
    input_size=gene_num,  
    hidden_size=hidden_size, 
    depth=depth,
    num_heads=head,
    classes=6, 
    mlp_ratio=4.0,
    dit_type='dit'
)

device = torch.device('mps')
model.to(device)

diffusion_step = diffusion_step


save_path_prefix = '/Users/derekebowman/Coding Projects/stDiff/stDiff/DEREK/example_results/demo.pt'

# Ensure the target folder exists:
os.makedirs(os.path.dirname(save_path_prefix), exist_ok=True)

model.train()

if not os.path.isfile(save_path_prefix):
    normal_train_stDiff(
        model,
        dataloader=dataloader,
        lr=lr,
        num_epoch=num_epoch,
        diffusion_step=diffusion_step,
        device=device,
        pred_type='noise',
        mask=mask
    )

    torch.save(model.state_dict(), save_path_prefix)
else:
    model.load_state_dict(torch.load(save_path_prefix, map_location=device))

# save_path_prefix = 'ckpt/demo.pt'
# # train
# model.train()
# if not os.path.isfile(save_path_prefix):

#     normal_train_stDiff(model,
#                             dataloader=dataloader,
#                             lr=lr,
#                             num_epoch=num_epoch,
#                             diffusion_step=diffusion_step,
#                             device=device,
#                             pred_type='noise',
#                             mask=mask)

#     torch.save(model.state_dict(), save_path_prefix)
# else:
#     model.load_state_dict(torch.load(save_path_prefix))



NameError: name 'NoiseScheduler' is not defined